In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException

In [4]:
state_links=["https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"]
state_name=['Kerala','Andra','Telungana','Goa','Rajasthan','South_Bengal','Hariyana','Assam','Uttra_Pradesh','West_Bengal']
state=dict(zip(state_name,state_links))

In [5]:
state

{'Kerala': 'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
 'Andra': 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile',
 'Telungana': 'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
 'Goa': 'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
 'Rajasthan': 'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',
 'South_Bengal': 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
 'Hariyana': 'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
 'Assam': 'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
 'Uttra_Pradesh': 'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile',
 'West_Bengal': 'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile'}

In [6]:
# Function to retrieve bus links and routes
def link_route(St_link):
    
    # Initialize WebDriver   
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
   
    Total_Links= []
    Total_Routs= []
    
    driver.get(St_link)
    # WebDriverWait instance
    wait = WebDriverWait(driver, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver.find_elements(By.XPATH,"//a[@class='route']")   #here path variable is used for getting arguments and the parameter is fixed ...it returns the element which contains the text and link
            for links in paths:  #one page contains many anchor tags with class = "routes" from that we traverse using 'links' variable 
                d = links.get_attribute("href")  #get_attribute is used to get the value used inside the html attribute...it is where we have retrieved link from .
                if d:
                    Total_Links.append(d)  #d is the link now...we have appended/added in the LINKS_KERALA list
            
            for route in paths:
                    Total_Routs.append(route.text)  #similarly for each anchor tag class='route', we are retrieving the route name by using route.text where .text is used to get value from the element
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']") 
                active_page_number = active_page_element.text #the pagination has numbers in it...that number is stored by using .text in active_page_number 
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"  #text() function is used to match the content with which we provided in the descendent of the div[class = " DC_117_pagination"]
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))  #we have to wait untill the webdriver loads the nextpage's XPATH
                
                # Scroll into view and click using JavaScript if necessary
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click() #we waited until next page xpath loaded...now after loaded we click
                except ElementNotInteractableException:
                    driver.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content  #this runs in loop...each page loads its data in 'try'  and then after finishing another page loads
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found") 
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    # Close the WebDriver  break
    driver.quit()
    return Total_Links, Total_Routs

In [7]:
for St_name,St_link in state.items():
    print(St_name)
    print(St_link)
   

Kerala
https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Andra
https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Telungana
https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
Goa
https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile
Rajasthan
https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile
South_Bengal
https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile
Hariyana
https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile
Assam
https://www.redbus.in/online-booking/astc/?utm_source=rtchometile
Uttra_Pradesh
https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile
West_Bengal
https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile


In [8]:
for St_name,St_link in state.items():
    print('State_link;',St_link)
    Total_Links,Total_Routs=link_route(St_link)
    df = pd.DataFrame({"Route_name":Total_Routs,"Route_link":Total_Links})
    df.to_csv(f"Data_Frame_{St_name}.csv", index=False)
    print("###################################################")

State_link; https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Navigating to page 2
No more pages to paginate or pagination element not found
###################################################
State_link; https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found
###################################################
State_link; https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found
###################################################
State_link; https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile
Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found
###################################################
State_

In [9]:
df

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
6,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...
7,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
8,Barasat (West Bengal) to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...


In [10]:
import pandas as pd
def Bus_Details(df):
    #retrive the bus details
    driver = webdriver.Chrome()
    Bus_names_k = []
    Bus_types_k = []
    Departure_k = []
    Arrival_k = []

    Ratings_k = []
    Total_Duration_k = []
    Prices_k = []
    Seats_Available_k = []
    Route_names = []
    Route_links = []

    #df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
    for i,r in df.iterrows(): 
        link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
        routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

    # Loop through each link
        driver.get(link)  #in particular bus link we have to extract...so to load that page we use get()
        time.sleep(2)   #time for waiting before interacting and executing next step dynamically

        # Click on elements to reveal bus details
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
        for element in elements: 
            element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
            time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
            
        # click elements to views bus
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
            clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
        except:
            continue  
        time.sleep(2)
        
        scrolling = True
        while scrolling:
            old_page_source = driver.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
            
            # Use ActionChains to perform a PAGE_DOWN
            ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                        #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                        #perform - executes the simulation
                                                                        
            
            time.sleep(5)  
            
            new_page_source = driver.page_source   #DOM Element is stored as string in new_page_source
            if new_page_source == old_page_source:
                scrolling = False                    #if new DOM element string is same as old DOM element string

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        try:
            rating = driver.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        # Append data to respective lists
        for bus in bus_name:
            Bus_names_k.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bus_types_k.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Departure_k.append(start_time_elem.text)
        for end_time_elem in end_time:
            Arrival_k.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_Duration_k.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_k.append(ratings.text)
        for price_elem in price:
            Prices_k.append(price_elem.text)
        for seats_elem in seats:
            Seats_Available_k.append(seats_elem.text)
    data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
    }   
    # from dictionary to convert data frame
    df_bus = pd.DataFrame(data)
    return df_bus


In [11]:
import os

csv_files = [f for f in os.listdir("Bus_Data") if f.endswith('.csv')]
#print(csv_files[0])
for i in range(0,len(csv_files)):
    df = pd.read_csv(os.path.join("Bus_Data", csv_files[i]))
    df_BUS=Bus_Details(df)
    df_BUS.to_csv(fr"./Bus_details/df_BUS{i}.csv",index=False)
print("Completed Successfully")    

Completed Successfully


In [6]:
import pandas as pd
df_BUS1=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS0.csv")
df_BUS2=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS1.csv")
df_BUS3=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS2.csv")
df_BUS4=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS3.csv")
df_BUS5=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS4.csv")
df_BUS6=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS5.csv")
df_BUS7=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS6.csv")
df_BUS8=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS7.csv")
df_BUS9=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS8.csv")
df_BUS10=pd.read_csv("E:/Data Science/RedBus_Project/Bus_details/df_BUS9.csv")

In [7]:
for i in range(len(df_BUS1)):
    df_BUS1['State']='Andhra Pradesh'
for i in range(len(df_BUS2)):
    df_BUS2['State']='Haryana'
for i in range(len(df_BUS3)):
    df_BUS3['State']='Karnataka'
for i in range(len(df_BUS4)):
    df_BUS4['State']='Kadamba'
for i in range(len(df_BUS5)):
    df_BUS5['State']='Rajasthan'
for i in range(len(df_BUS6)):
    df_BUS6['State']='South Bengal'
for i in range(len(df_BUS7)):
    df_BUS7['State']='Telangana'
for i in range(len(df_BUS8)):
    df_BUS8['State']='Uttar pradesh'
for i in range(len(df_BUS9)):
    df_BUS9['State']='West Bengal'
for i in range(len(df_BUS10)):
    df_BUS10['State']='Goa'

In [8]:
print(df_BUS1.shape)
print(df_BUS2.shape)
print(df_BUS3.shape)
print(df_BUS4.shape)
print(df_BUS5.shape)
print(df_BUS6.shape)
print(df_BUS7.shape)
print(df_BUS8.shape)
print(df_BUS9.shape)
print(df_BUS10.shape)

(709, 11)
(0, 10)
(22, 11)
(287, 11)
(244, 11)
(227, 11)
(69, 11)
(350, 11)
(379, 11)
(26, 11)


In [9]:
df_Buses=pd.concat([df_BUS1,df_BUS2,df_BUS3,df_BUS4,df_BUS5,df_BUS6,df_BUS7,df_BUS8,df_BUS9,df_BUS10])

C:\Users\kumar\AppData\Local\Temp\ipykernel_3080\648495311.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Buses=pd.concat([df_BUS1,df_BUS2,df_BUS3,df_BUS4,df_BUS5,df_BUS6,df_BUS7,df_BUS8,df_BUS9,df_BUS10])


In [10]:
df_Buses['State'].unique()

array(['Andhra Pradesh', 'Karnataka', 'Kadamba', 'Rajasthan',
       'South Bengal', 'Telangana', 'Uttar pradesh', 'West Bengal', 'Goa'],
      dtype=object)

In [11]:
df_Buses['Seats_Available']=df_Buses['Seats_Available'].str.split(' ').str[0]

In [12]:
df_Buses['Seats_Available']=pd.to_numeric(df_Buses['Seats_Available'],errors='coerce')

In [13]:
print(df_Buses['Seats_Available'].unique())

[19 24 22 26 20 17 34 31 25 33 35 27 30  1 38 36 47 43 39 42 21 29 45 28
 14 18 44 37 32  4 13  6 10 16 23 15  7  9 11  8  5 12 40 41  2  3 46 49
 48 50 55 52 51]


In [14]:
df_Buses['Ratings']=df_Buses['Ratings'].str.split('\n').str[0]
df_Buses['Ratings'] = pd.to_numeric(df_Buses['Ratings'], errors='coerce')

In [15]:
df_Buses['Ratings'].unique()

array([4.7, 4.6, 4.1, 4.8, 4.5, 3.8, 3.3, 3.9, 3.4, 2.9, 4. , 4.3, 3.6,
       nan, 3.5, 4.4, 3.7, 1.7, 2.8, 3. , 1.8, 4.9, 4.2, 2.3, 2. , 2.7,
       2.5, 3.1, 2.6, 1.3, 2.1, 3.2, 5. , 1.9, 2.4, 2.2, 1.4, 1.6, 1. ,
       1.5])

In [16]:
df_Buses['Ratings']=df_Buses['Ratings'].fillna(df_Buses['Ratings'].median())

In [17]:
df_Buses['From']=df_Buses['Route_name'].str.split('to').str[0]
df_Buses['To']=df_Buses['Route_name'].str.split('to').str[1]

In [18]:
df_Buses=df_Buses.drop('Route_name',axis=1)

In [19]:
print(df_Buses.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2313 entries, 0 to 25
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         2313 non-null   object 
 1   Bus_type         2313 non-null   object 
 2   Start_time       2313 non-null   object 
 3   End_time         2313 non-null   object 
 4   Total_duration   2313 non-null   object 
 5   Price            2313 non-null   float64
 6   Seats_Available  2313 non-null   int64  
 7   Ratings          2313 non-null   float64
 8   Route_link       2313 non-null   object 
 9   State            2313 non-null   object 
 10  From             2313 non-null   object 
 11  To               2313 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 234.9+ KB
None


In [20]:
print(df_Buses.head())

            Bus_name                               Bus_type Start_time  \
0  IntrCity SmartBus             A/C Seater / Sleeper (2+1)      23:25   
1  IntrCity SmartBus  Bharat Benz A/C Seater /Sleeper (2+1)      23:45   
2  IntrCity SmartBus             A/C Seater / Sleeper (2+1)      22:45   
3   AdSai RK Travels         Non A/C Seater / Sleeper (2+1)      22:10   
4  IntrCity SmartBus                       AC Sleeper (2+1)      23:45   

  End_time Total_duration  Price  Seats_Available  Ratings  \
0    06:15        06h 50m  499.0               19      4.7   
1    06:00        06h 15m  529.0               24      4.6   
2    05:45        07h 00m  499.0               22      4.6   
3    05:30        07h 20m  580.0               19      4.1   
4    06:00        06h 15m  779.0               19      4.8   

                                          Route_link           State  \
0  https://www.redbus.in/bus-tickets/vijayawada-t...  Andhra Pradesh   
1  https://www.redbus.in/bus-tickets/v

In [21]:
import pymysql
Database = pymysql.connect(host='127.0.0.1',user='root',passwd='161121',database='redbus')
cursor=Database.cursor()

In [22]:
cursor.execute('''
                CREATE TABLE IF NOT EXISTS busdetails (
                    State VARCHAR(255),
                    Bus_name VARCHAR(255),
                    Bus_type VARCHAR(255),
                    Starting_time VARCHAR(255),
                    Ending_time VARCHAR(255),
                    Duration VARCHAR(255),
                    Seats_available INT,
                    Ratings FLOAT,
                    Price DECIMAL(10, 2),
                    From_point VARCHAR(255),
                    To_point VARCHAR(255),
                    Route_link VARCHAR(255));               
''')

0

In [23]:
Sql_query='''INSERT INTO busdetails(State,Bus_name,Bus_type,Starting_time,Ending_time,Duration,Seats_available,Ratings,Price,From_point,To_point,Route_link) 
                    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for index,rows in df_Buses.iterrows():
    cursor.execute(
        Sql_query,(
            rows['State'],
            rows['Bus_name'],
            rows['Bus_type'],
            rows['Start_time'],
            rows['End_time'],
            rows['Total_duration'],
            rows['Seats_Available'],
            rows['Ratings'],
            rows['Price'],
            rows['From'],
            rows['To'],
            rows['Route_link']
        ))
Database.commit()

In [24]:
df_Buses.to_excel('Cleaned_Dataset.xlsx',index=False)

In [25]:
print(df_Buses[df_Buses['State']=='Andhra Pradesh'])

              Bus_name                               Bus_type Start_time  \
0    IntrCity SmartBus             A/C Seater / Sleeper (2+1)      23:25   
1    IntrCity SmartBus  Bharat Benz A/C Seater /Sleeper (2+1)      23:45   
2    IntrCity SmartBus             A/C Seater / Sleeper (2+1)      22:45   
3     AdSai RK Travels         Non A/C Seater / Sleeper (2+1)      22:10   
4    IntrCity SmartBus                       AC Sleeper (2+1)      23:45   
..                 ...                                    ...        ...   
704       Indu Travels         Non A/C Seater / Sleeper (2+1)      22:00   
705            PRL BUS                      A/C Sleeper (2+1)      21:35   
706       Indu Travels         Non A/C Seater / Sleeper (2+1)      21:45   
707          INDHU BUS         Non A/C Seater / Sleeper (2+1)      21:30   
708       Indu Travels                      A/C Sleeper (2+1)      21:35   

    End_time Total_duration  Price  Seats_Available  Ratings  \
0      06:15        06h